In [1]:
import yt
from yt import YTArray
from yt import YTQuantity
from yt.data_objects.level_sets.api import *

from astropy import constants as const

import numpy as np

import matplotlib
%matplotlib inline
import matplotlib.pylab as plt
from matplotlib.colors import SymLogNorm, LogNorm

import palettable
import plotting_tools as pt
import yt_functions as ytf





yt : [INFO     ] 2020-01-22 13:42:58,136 Parameters: current_time              = 0
yt : [INFO     ] 2020-01-22 13:42:58,138 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2020-01-22 13:42:58,139 Parameters: domain_left_edge          = [-3. -3. -3.]
yt : [INFO     ] 2020-01-22 13:42:58,140 Parameters: domain_right_edge         = [3. 3. 3.]
yt : [INFO     ] 2020-01-22 13:42:58,141 Parameters: cosmological_simulation   = 0.0


In [20]:
def get_masked_data_source(ds, z_min = 0.8, z_max = 1.2, T_cold = 3.33333e5):
    ad = ds.all_data()
    z_abs_code = np.abs(ad[('gas', 'z')] / ds.length_unit.in_units('kpc'))
    zmask= (z_abs_code >= z_min) & (z_abs_code <= z_max)
    return ad[zmask]

def _maximum_gas_temperature(clump, max_temp):
    return (clump["gas", "temperature"].sum() <= max_temp)
add_validator("maximum_gas_temperature", _maximum_gas_temperature)

In [3]:
i = 50
folder = 'production/isocool_tctf_0.3_beta_100.0'
ds = ytf.load('../../simulations/%s/DD%04d/DD%04d'%(folder, i, i))


yt : [INFO     ] 2020-01-22 13:43:26,315 Parameters: current_time              = 1.4999997187781
yt : [INFO     ] 2020-01-22 13:43:26,316 Parameters: domain_dimensions         = [ 64  64 256]
yt : [INFO     ] 2020-01-22 13:43:26,317 Parameters: domain_left_edge          = [-0.5 -0.5 -2. ]
yt : [INFO     ] 2020-01-22 13:43:26,318 Parameters: domain_right_edge         = [0.5 0.5 2. ]
yt : [INFO     ] 2020-01-22 13:43:26,319 Parameters: cosmological_simulation   = 0.0
yt : [INFO     ] 2020-01-22 13:43:26,323 Gathering a field list (this may take a moment.)


In [34]:
ad = ds.all_data()
reg = ds.r[:, :, .8:1.2]


In [29]:
master_clump = Clump(reg, ('gas', 'temperature'))
master_clump.add_validator("min_cells", 8)
#master_clump.add_validator("maximum_gas_temperature", 3.33333e5)



In [30]:
c_min = reg["gas", "temperature"].min()
c_max = reg["gas", "temperature"].max()
step = 2.0
find_clumps(master_clump, c_min, c_max, step)

yt : [INFO     ] 2020-01-22 13:58:54,753 Finding clumps: min: 4.893958e+04, max: 2.119404e+06, step: 2.000000
yt : [INFO     ] 2020-01-22 13:58:55,064 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 13:58:55,450 Linked.
yt : [INFO     ] 2020-01-22 13:58:56,436 Finding clumps: min: 9.787916e+04, max: 2.119404e+06, step: 2.000000
yt : [INFO     ] 2020-01-22 13:58:56,526 Wiping out existing children clumps: 1.
yt : [INFO     ] 2020-01-22 13:58:56,862 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 13:58:57,186 Linked.
yt : [INFO     ] 2020-01-22 13:58:58,036 Finding clumps: min: 1.957583e+05, max: 2.119404e+06, step: 2.000000
yt : [INFO     ] 2020-01-22 13:58:58,037 Wiping out existing children clumps: 1.
yt : [INFO     ] 2020-01-22 13:58:58,454 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 13:58:58,799 Linked.
yt : [INFO     ] 2020-01-22 13:59:00,028 Finding clumps: min: 3.915167e+05, max: 2.119404e+06, step: 2.000000
yt : [INFO     ] 2020-01-22 13:59:00,030 Wiping

In [33]:
master_clump.add_info_item("total_cells")
leaf_clumps = master_clump.leaves
clump = leaf_clumps[1]
print(clump.info['total_cells'])

('Cells: %d.', 9)


In [17]:
print(master_clump["gas", "temperature"])
print(master_clump.children)
print(master_clump.children[0]["gas", "temperature"])

[1629145.27549999 1622939.90733232 1609259.02952231 ...   56302.5848589
   57672.93987246   60644.63773788] K
[<yt.data_objects.level_sets.clump_handling.Clump object at 0x2aaae52c5908>, <yt.data_objects.level_sets.clump_handling.Clump object at 0x2aaae52d4e80>]
[1629145.27549999 1622939.90733232 1609259.02952231 ... 1751280.29349854
 1744215.69161679 1689307.98969373] K


In [18]:
for clump in master_clump.tree:
    print(clump.clump_id)

AttributeError: 'Clump' object has no attribute 'tree'

In [41]:
data_source = ds.r[:, :, .8:1.2]
field = ("gas", "density")
step = 2.0
c_min = 10**np.floor(np.log10(data_source[field]).min()  )
c_max = 10**np.floor(np.log10(data_source[field]).max()+1)
master_clump = Clump(data_source, field)
master_clump.add_info_item("center_of_mass")
master_clump.add_validator("min_cells", 8)
find_clumps(master_clump, c_min, c_max, step)
fn = master_clump.save_as_dataset(fields=["density", "cell_mass", "temperature"])
new_ds = yt.load(fn)
print(new_ds.tree["clump", "cell_mass"])
print(new_ds.tree["grid", "density"])
print(new_ds.tree["all", "particle_mass"])
print(new_ds.tree.children[0]["clump", "cell_mass"])
print(new_ds.leaves[0]["clump", "cell_mass"])
print(new_ds.leaves[0]["grid", "density"])

yt : [INFO     ] 2020-01-22 14:46:56,828 Finding clumps: min: 1.000000e-28, max: 1.000000e-26, step: 2.000000
yt : [INFO     ] 2020-01-22 14:46:57,364 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 14:46:57,749 Linked.
yt : [INFO     ] 2020-01-22 14:47:01,831 Finding clumps: min: 2.000000e-28, max: 1.000000e-26, step: 2.000000
yt : [INFO     ] 2020-01-22 14:47:01,927 Wiping out existing children clumps: 1.
yt : [INFO     ] 2020-01-22 14:47:02,361 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 14:47:02,729 Linked.
yt : [INFO     ] 2020-01-22 14:47:06,933 Finding clumps: min: 4.000000e-28, max: 1.000000e-26, step: 2.000000
yt : [INFO     ] 2020-01-22 14:47:07,029 Wiping out existing children clumps: 1.
yt : [INFO     ] 2020-01-22 14:47:07,540 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 14:47:07,974 Linked.
yt : [INFO     ] 2020-01-22 14:47:38,930 Investigating 11 children.
yt : [INFO     ] 2020-01-22 14:47:39,028 Finding clumps: min: 8.000000e-28, max: 1.000000

yt : [INFO     ] 2020-01-22 14:49:52,228 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 14:49:52,609 Linked.
yt : [INFO     ] 2020-01-22 14:49:59,131 Investigating 2 children.
yt : [INFO     ] 2020-01-22 14:49:59,229 Finding clumps: min: 1.600000e-27, max: 1.000000e-26, step: 2.000000
yt : [INFO     ] 2020-01-22 14:50:01,542 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 14:50:01,930 Linked.
yt : [INFO     ] 2020-01-22 14:50:05,733 Finding clumps: min: 3.200000e-27, max: 1.000000e-26, step: 2.000000
yt : [INFO     ] 2020-01-22 14:50:05,830 Wiping out existing children clumps: 1.
yt : [INFO     ] 2020-01-22 14:50:07,143 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 14:50:07,658 Linked.
yt : [INFO     ] 2020-01-22 14:50:11,629 Finding clumps: min: 6.400000e-27, max: 1.000000e-26, step: 2.000000
yt : [INFO     ] 2020-01-22 14:50:11,632 Wiping out existing children clumps: 1.
yt : [INFO     ] 2020-01-22 14:50:12,940 Linking node (1) contours.
yt : [INFO     ] 2020-

yt : [INFO     ] 2020-01-22 14:52:21,232 Wiping out existing children clumps: 1.
yt : [INFO     ] 2020-01-22 14:52:22,635 Linking node (1) contours.
yt : [INFO     ] 2020-01-22 14:52:23,023 Linked.
yt : [INFO     ] 2020-01-22 14:52:32,331 Investigating 3 children.
yt : [INFO     ] 2020-01-22 14:52:32,427 Finding clumps: min: 1.280000e-26, max: 1.000000e-26, step: 2.000000
yt : [INFO     ] 2020-01-22 14:52:32,428 Finding clumps: min: 1.280000e-26, max: 1.000000e-26, step: 2.000000
yt : [INFO     ] 2020-01-22 14:52:32,429 Eliminating invalid, childless clump with 2 cells.
yt : [INFO     ] 2020-01-22 14:52:32,429 Finding clumps: min: 1.280000e-26, max: 1.000000e-26, step: 2.000000
yt : [INFO     ] 2020-01-22 14:52:32,430 Eliminating invalid, childless clump with 1 cells.
yt : [INFO     ] 2020-01-22 14:52:32,431 1 of 3 children survived, linking its children to parent.
yt : [INFO     ] 2020-01-22 14:52:32,526 10 of 11 children survived.
yt : [INFO     ] 2020-01-22 14:52:35,727 Saving field

AttributeError: 'EnzoDataset' object has no attribute 'tree'

In [43]:
print(new_ds.tree["clump", "cell_mass"])
print(new_ds.tree["grid", "density"])
print(new_ds.tree["all", "cell_mass"])
print(new_ds.tree.children[0]["clump", "cell_mass"])
print(new_ds.leaves[0]["clump", "cell_mass"])
print(new_ds.leaves[0]["grid", "density"])

242533823.81594494 Msun
[2.69104048e-28 2.66132733e-28 2.64875506e-28 ... 3.59263201e-27
 2.81528480e-27 2.00404559e-27] g/cm**3


KeyError: "Unable to open object (object 'all' doesn't exist)"